## Setup

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

from harp_resources import process, utils
from sleap import load_and_process as lp

In [ ]:
data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03')
photometry_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03/photometry')
SessionSettings = utils.read_SessionSettings(data_path, print_contents=False)

## Videography and SLEAP
### Load and QC videography data
commented out lines may be useful

In [ ]:
VideoData1, VideoData2, VideoData1_Has_Sleap, VideoData2_Has_Sleap = lp.load_videography_data(data_path)
VideoData1 = VideoData1.drop(columns=['track'])
columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']
coordinates_dict=lp.get_coordinates_dict(VideoData1, columns_of_interest)

In [ ]:
# Create figure for x coordinates
fig_x = go.Figure()
fig_x.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['left.x'], mode='lines', name='left.x'))
fig_x.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['center.x'], mode='lines', name='center.x'))
fig_x.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['right.x'], mode='lines', name='right.x'))
fig_x.update_layout(title='X coordinates for pupil centre add left-right eye corner', xaxis_title='Seconds', yaxis_title='X Position')
fig_x.show()

# Create figure for y coordinates
fig_y = go.Figure()
fig_y.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['left.y'], mode='lines', name='left.y'))
fig_y.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['center.y'], mode='lines', name='center.y'))
fig_y.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1['right.y'], mode='lines', name='right.y'))
fig_y.update_layout(title='Y coordinates for pupil centre add left-right eye corner', xaxis_title='Seconds', yaxis_title='Y Position')
fig_y.show()

# Create figure for p.x coordinates
fig_px = go.Figure()
for col in ['p1.x', 'p2.x', 'p3.x', 'p4.x', 'p5.x', 'p6.x', 'p7.x', 'p8.x']:
    fig_px.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1[col], mode='lines', name=col))
fig_px.update_layout(title='X coordinates for iris points', xaxis_title='Seconds', yaxis_title='X Position')
fig_px.show()

# Create figure for p.y coordinates
fig_py = go.Figure()
for col in ['p1.y', 'p2.y', 'p3.y', 'p4.y', 'p5.y', 'p6.y', 'p7.y', 'p8.y']:
    fig_py.add_trace(go.Scatter(x=VideoData1['Seconds'], y=VideoData1[col], mode='lines', name=col))
fig_py.update_layout(title='Y coordinates for iris points', xaxis_title='Seconds', yaxis_title='Y Position')
fig_py.show()

In [ ]:
columns_of_interest = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']

# Filter out NaN values and calculate the min and max values for X and Y coordinates
x_min = min([coordinates_dict[f'{col}.x'][~np.isnan(coordinates_dict[f'{col}.x'])].min() for col in columns_of_interest])
x_max = max([coordinates_dict[f'{col}.x'][~np.isnan(coordinates_dict[f'{col}.x'])].max() for col in columns_of_interest])
y_min = min([coordinates_dict[f'{col}.y'][~np.isnan(coordinates_dict[f'{col}.y'])].min() for col in columns_of_interest])
y_max = max([coordinates_dict[f'{col}.y'][~np.isnan(coordinates_dict[f'{col}.y'])].max() for col in columns_of_interest])

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 7))

# Plot left, right, and center in the first plot
ax[0].set_title('left, right, center')
ax[0].scatter(coordinates_dict['left.x'], coordinates_dict['left.y'], color='black', label='left', s=10)
ax[0].scatter(coordinates_dict['right.x'], coordinates_dict['right.y'], color='grey', label='right', s=10)
ax[0].scatter(coordinates_dict['center.x'], coordinates_dict['center.y'], color='red', label='center', s=10)
ax[0].set_xlim([x_min, x_max])
ax[0].set_ylim([y_min, y_max])
ax[0].set_xlabel('x coordinates (pixels)')
ax[0].set_ylabel('y coordinates (pixels)')
ax[0].legend(loc='upper right')

# Plot p1 to p8 in the second plot with different colors and smaller markers
colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'orange']
for idx, col in enumerate(columns_of_interest[3:]):
    ax[1].scatter(coordinates_dict[f'{col}.x'], coordinates_dict[f'{col}.y'], color=colors[idx], label=col, s=5)

ax[1].set_xlim([x_min, x_max])
ax[1].set_ylim([y_min, y_max])
ax[1].set_title('p1 to p8')
ax[1].set_xlabel('x coordinates (pixels)')
ax[1].set_ylabel('y coordinates (pixels)')
ax[1].legend(loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']

all_nan_df = VideoData1[VideoData1[columns_of_interest].isnull().all(1)]
all_nan_index_array = all_nan_df.index.values

# print the groups of sequential NaNs
group_counts = {'1-5': 0, '6-10': 0, '>10': 0}
i = 1
for group in lp.find_sequential_groups(all_nan_index_array):
    #print(f'NaN frame group {i} with {len(group)} elements')
    if 1 <= len(group) <= 5:
        group_counts['1-5'] += 1
    elif 6 <= len(group) <= 10:
        group_counts['6-10'] += 1
    else:
        group_counts['>10'] += 1
        print(f'Framegroup {i} has {len(group)} consecutive all NaN frames  with indices {group}')
    i += 1

print(f"Framegroups with 1-5 consecutive all NaN frames: {group_counts['1-5']}")
print(f"Framegroups with 6-10 consecutive all NaN frames: {group_counts['6-10']}")
print(f"Framegroups with >10 consecutive all NaN frames: {group_counts['>10']}")

### SLEAP processing

In [ ]:
# make mean pupil center coordinate zero 
columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']

# Calculate the mean of the center x and y points
mean_center_x = VideoData1['center.x'].mean()
mean_center_y = VideoData1['center.y'].mean()

print(f"Mean center.x: {mean_center_x}, Mean center.y: {mean_center_y}")

# Translate the coordinates
for col in columns_of_interest:
    if '.x' in col:
        VideoData1[col] = VideoData1[col] - mean_center_x
    elif '.y' in col:
        VideoData1[col] = VideoData1[col] - mean_center_y

# Calculate the standard deviation for each column of interest
std_devs = {col: VideoData1[col].std() for col in columns_of_interest}

# Calculate the number of outliers for each column
outliers = {col: ((VideoData1[col] - VideoData1[col].mean()).abs() > 10 * std_devs[col]).sum() for col in columns_of_interest}

# Find the channel with the maximum number of outliers
max_outliers_channel = max(outliers, key=outliers.get)
max_outliers_count = outliers[max_outliers_channel]

# Print the channel with the maximum number of outliers and the number
print(f"Channel with the maximum number of outliers: {max_outliers_channel}, Number of outliers: {max_outliers_count}")

# Print the total number of outliers
total_outliers = sum(outliers.values())
print(f"A total number of {total_outliers} outliers will be replaced by interpolation")

# Replace outliers by interpolating between the previous and subsequent non-NaN value
for col in columns_of_interest:
    outlier_indices = VideoData1[((VideoData1[col] - VideoData1[col].mean()).abs() > 10 * std_devs[col])].index
    VideoData1.loc[outlier_indices, col] = np.nan

VideoData1.interpolate(inplace=True)

In [ ]:
columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']
coordinates_dict=lp.get_coordinates_dict(VideoData1, columns_of_interest)

columns_of_interest = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']

# Filter out NaN values and calculate the min and max values for X and Y coordinates
x_min = min([coordinates_dict[f'{col}.x'][~np.isnan(coordinates_dict[f'{col}.x'])].min() for col in columns_of_interest])
x_max = max([coordinates_dict[f'{col}.x'][~np.isnan(coordinates_dict[f'{col}.x'])].max() for col in columns_of_interest])
y_min = min([coordinates_dict[f'{col}.y'][~np.isnan(coordinates_dict[f'{col}.y'])].min() for col in columns_of_interest])
y_max = max([coordinates_dict[f'{col}.y'][~np.isnan(coordinates_dict[f'{col}.y'])].max() for col in columns_of_interest])

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 7))

# Plot left, right, and center in the first plot
ax[0].set_title('left, right, center')
ax[0].scatter(coordinates_dict['left.x'], coordinates_dict['left.y'], color='black', label='left', s=10)
ax[0].scatter(coordinates_dict['right.x'], coordinates_dict['right.y'], color='grey', label='right', s=10)
ax[0].scatter(coordinates_dict['center.x'], coordinates_dict['center.y'], color='red', label='center', s=10)
ax[0].set_xlim([x_min, x_max])
ax[0].set_ylim([y_min, y_max])
ax[0].set_xlabel('x coordinates (pixels)')
ax[0].set_ylabel('y coordinates (pixels)')
ax[0].legend(loc='upper right')

# Plot p1 to p8 in the second plot with different colors and smaller markers
colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'orange']
for idx, col in enumerate(columns_of_interest[3:]):
    ax[1].scatter(coordinates_dict[f'{col}.x'], coordinates_dict[f'{col}.y'], color=colors[idx], label=col, s=5)

ax[1].set_xlim([x_min, x_max])
ax[1].set_ylim([y_min, y_max])
ax[1].set_title('p1 to p8')
ax[1].set_xlabel('x coordinates (pixels)')
ax[1].set_ylabel('y coordinates (pixels)')
ax[1].legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
VideoData1 = VideoData1.interpolate(method='linear', limit_direction='both')

columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']
coordinates_dict=lp.get_coordinates_dict(VideoData1, columns_of_interest)

theta = lp.find_horizontal_axis_angle(VideoData1, 'left', 'center')
center_point = lp.get_left_right_center_point(coordinates_dict)

columns_of_interest = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
remformatted_coordinates_dict = lp.get_reformatted_coordinates_dict(coordinates_dict, columns_of_interest)
centered_coordinates_dict = lp.get_centered_coordinates_dict(remformatted_coordinates_dict, center_point)
rotated_coordinates_dict = lp.get_rotated_coordinates_dict(centered_coordinates_dict, theta)

columns_of_interest = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
ellipse_parameters_data, ellipse_center_points_data = lp.get_fitted_ellipse_parameters(rotated_coordinates_dict, columns_of_interest)

average_diameter = np.mean([ellipse_parameters_data[:,0], ellipse_parameters_data[:,1]], axis=0)

SleapVideoData1 = process.convert_arrays_to_dataframe(['Seconds', 'Ellipse.Diameter', 'Ellipse.Angle', 'Ellipse.Center.X', 'Ellipse.Center.Y'], [VideoData1['Seconds'].values, average_diameter, ellipse_parameters_data[:,2], ellipse_center_points_data[:,0], ellipse_center_points_data[:,1]])

# NOW this is great. 
### 1st - move to functions, make it print a figure of the original and cleaned up coordinate distribution data. PLUS the cleaned up timeseries data. 
### 2nd - remove video Data loading from loading notebook and insert these functions instead.
### 3nd - get the videodata, make datetime indexed (aeon?) and insert into the padded all data dataframe. Not sure how will it behave with resampling, need to think about it. 

In [ ]:
pympler_memory_df = utils.get_pympler_memory_usage(top_n=100)
